## <font color='red'> INSTRUCTIONS </font>

<b> 
1. Write your code only in cells below the "WRITE CODE BELOW" title. Do not modify the code below the "DO NOT MODIFY" title. <br>
2. The expected data types of the output answers for each question are given in the last cell through assertion statements. Your answers must match these expected output data types. Hint: Many of the answers need to be a Python dictionary. Consider methods like to_dict() to convert a Pandas Series to a dictionary. <br>
3. The answers are then written to a JSON file named my_results_PA1.json. You can compare this with the provided expected output file "expected_results_PA1.json". <br>
4. After you complete writing your code, click "Kernel -> Restart Kernel and Run All Cells" on the top toolbar. There should NOT be any syntax/runtime errors, otherwise points will be deducted. <br>
5. For submitting your solution, first download your notebook by clicking "File -> Download". Rename the file as &ltTEAM_ID&gt.ipynb" and upload to Canvas.</b>


## <font color='red'> DO NOT MODIFY </font>

In [3]:
import time
import json
import dask
import dask.dataframe as dd
import pandas as pd
import ast
import re
from dask.distributed import Client
import ctypes
import numpy as np

def trim_memory() -> int:
    """
    helps to fix any memory leaks.
    """
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)

# change client url to the dask worker url from one of our 5 billion terminals ig
client = Client("172.31.35.90:8786")
client.run(trim_memory)
client = client.restart()
print(client)

/home/ubuntu/dask_env/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40937 instead
  warnings.warn(
2025-05-09 09:46:45,880 - distributed.nanny - WARNING - Restarting worker
2025-05-09 09:46:45,887 - distributed.nanny - WARNING - Restarting worker
2025-05-09 09:46:45,891 - distributed.nanny - WARNING - Restarting worker
2025-05-09 09:46:45,896 - distributed.nanny - WARNING - Restarting worker


None


2025-05-09 09:51:07,006 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:45761' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('read-_to_string_dtype-lambda-07ae5c08706f590f8fa7c977f48b1694', 271), ('read-_to_string_dtype-lambda-07ae5c08706f590f8fa7c977f48b1694', 36), ('read-_to_string_dtype-lambda-07ae5c08706f590f8fa7c977f48b1694', 289), ('read-_to_string_dtype-lambda-07ae5c08706f590f8fa7c977f48b1694', 228), ('read-_to_string_dtype-lambda-07ae5c08706f590f8fa7c977f48b1694', 54), ('read-_to_string_dtype-lambda-07ae5c08706f590f8fa7c977f48b1694', 292), ('read-_to_string_dtype-lambda-07ae5c08706f590f8fa7c977f48b1694', 231), ('read-_to_string_dtype-lambda-07ae5c08706f590f8fa7c977f48b1694', 2), ('read-_to_string_dtype-lambda-07ae5c08706f590f8fa7c977f48b1694', 246), ('read-_to_string_dtype-lambda-07ae5c08706f590f8fa7c977f48b1694', 69), ('read-_to_string_dtype-lambda-07ae5c08706f590f8fa7c977f48b1694', 8), ('read-_to_string_dtype-l

In [4]:
start = time.time()

## <font color='blue'> WRITE CODE BELOW </font>

In [5]:
user_reviews = dd.read_csv('user_reviews.csv')
products = dd.read_csv('products.csv')

# Hints

1. We advise you to start using 5 instances so that you can test your code faster. This aligns with the “fail-fast”
philosophy. Once your code is working you can measure the runtime on two and five instances and make
further optimizations.
2. What are dangling references?
If there is a value V in column X of table A, but this value is missing in column X of table B, we say that
V is a dangling reference of X from A to B.
For example, consider the below two tables:
ID X
1 21
2 32
3 1
ID Y
1 4
3 5
Here, ID 2 is a dangling reference from the first table to the second.
3. Worker logs will be output to the terminals where you started your workers and may not appear in jupyter-
notebook. You will need to check the terminal output for your workers to debug any errors.
2


# Q1: Get percentage of missing values for all columns in the reviews table.
## Output must be a dictionary with col names as keys and percentages as values
Idea: iterate through each column, calling count na and then dividing by dataframe shape?
- Not sure how this works with partitioning

In [15]:
user_reviews.map_partitions(lambda x: x.isna().sum()).compute()

reviewerID           0
asin                 0
reviewerName      3726
helpful              0
reviewText          20
overall              0
summary             13
unixReviewTime       0
reviewTime           0
dtype: int64

# Q2. Get percentage of missing values for all columns in the products table.
## For output type see Q1
Idea: iterate through each column, calling count na and then dividing by dataframe shape

In [ ]:
products.map_partitions(lambda x: x.isna().sum()).compute()

# Q3. Find Pearson correlation coefficient between the price and rating of the products.
## Output must be a float
Idea: grab price column from the product table and the rating column from the review table then calculate Pearson
- Pearson correlation formula: sum(x_i - x_bar)(y_i - y_bar)/square root(sum(x_i-x_bar)^2 * sum(y_i - y_bar)^2)

# Q4. Find mean, standard deviation, median, min, and max for the price column in the products table.
## Output must be a dictionary
Idea: 
- Grab the price column from the products table
- Calculate each of the metrics by calling pandas on the partitions???
- Not sure the exact format but we'll find out

# Q5. Find number of products for each super-category (the first entry in the “categories” column in the products table). Output categories should be sorted in non-increasing order in the number of products. Categories with the same number of products can appear in any order.
## Output should be a dictionary
Idea: 
- Process the category column from the products table and get first value from each list
- Groupby this processed column and use .count()
- Sort in non-increasing order

# Q6. Check (return 1 or 0) if there are any dangling references from product ids in the reviews table to products table. Return 1 if there are dangling references and 0 otherwise.
## Output should be a Boolean
Idea:
- Join??? might be super inefficient
- Get the number of unique product ids in reviews and see if == number of unique product ids in products
- This is the asin column btw

# Q7. Check (return 1 or 0) if there are any dangling references from product ids in the “related” column to the “asin” column of the product table. Return 1 if there are dangling references and 0 otherwise.
## Output should be a Boolean
Idea: 
- So I think this question is asking if there are asins that are not in any of the related columns
- Process the related column in the product table, which is currently a dictionary where keys are 'Also viewed', 'Bought after', etc. and entries are lists of asins
- Going to have to use a lambda function on the dict
- Kinda iffy idea: iterate through the related column, adding unique ids to a list and seeing if the length of .unique() of that == length of .unique() on asins

In [ ]:
### read in the 'user_reviews.csv' and 'products.csv' files, perform your calculations and place the answers in variables ans1 - ans7.


# substitute 'None' with the outputs from your calculations. 
# The expected output types can be seen in the assertion statements below
ans1 = None
ans2 = None
ans3 = None
ans4 = None
ans5 = None
ans6 = None
ans7 = None

## <font color='red'> DO NOT MODIFY </font>

In [ ]:
end = time.time()

In [ ]:
print(f"execution time = {end-start}s")

In [ ]:
# DO NOT MODIFY
assert type(ans1) == dict, f"answer to question 1 must be a dictionary like {{'reviewerID':0.2, ..}}, got type = {type(ans1)}"
assert type(ans2) == dict, f"answer to question 2 must be a dictionary like {{'asin':0.2, ..}}, got type = {type(ans2)}"
assert type(ans3) == float, f"answer to question 3 must be a float like 0.8, got type = {type(ans3)}"
assert type(ans4) == dict, f"answer to question 4 must be a dictionary like {{'mean':0.4,'max':0.6,'median':0.6...}}, got type = {type(ans4)}"
assert type(ans5) == dict, f"answer to question 5 must be a dictionary, got type = {type(ans5)}"         
assert ans6 == 0 or ans6==1, f"answer to question 6 must be 0 or 1, got value = {ans6}" 
assert ans7 == 0 or ans7==1, f"answer to question 7 must be 0 or 1, got value = {ans7}" 

ans_dict = {
    "q1": ans1,
    "q2": ans2,
    "q3": ans3,
    "q4": ans4,
    "q5": ans5,
    "q6": ans6,
    "q7": ans7,
    "runtime": end-start
}
with open('my_results_PA1.json', 'w') as outfile: json.dump(ans_dict, outfile)         